In [6]:
# Simple interactive Python debugger

# Our buggy program
def remove_html_markup(s):
    tag   = False
    quote = False
    out   = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c
    return out
    
# main program that runs the buggy program
def main():
    print remove_html_markup('xyz')
    print remove_html_markup('"<b>foo</b>"')
    print remove_html_markup("'<b>foo</b>'")

    
# ------------ my-spyder --------------------------
""" *** INSTRUCTIONS ***
1. Improve and expand the 'debug' function to accept a print command 'p <arg>'.
If the print command has no argument, print out the dictionary that 
holds all variables.
Print the value of the supplied variable in a form 'var = repr(value)', 
if the 'p' is followed by an argument, or print 'No such variable:', arg 
if no such variable exists.

2. Improve and expand the 'debug' function to accept a breakpoint 
command 'b <line>'.
Add the line number to the breakpoints dictionary, or print 
'You must supply a line number' if 'b' is not followed by a line number.

3. Improve and expand the 'debug' function to accept a watchpoint 
command 'w <var name>'.
Add the variable name to the watchpoints dictionary, or print 
'You must supply a variable name' if 'w' is not followed by a string.
"""

import sys
  
# globals
stepping = False
breakpoints = {}
watchpoints = {'out': None}

def debug(command, my_locals):
    global stepping
    global breakpoints
    global watchpoints
    
    if command.find(' ') > 0:
        arg = command.split(' ')[1]
    else:
        arg = None

    if command.startswith('s'):       # step
        stepping = True
        return True
    elif command.startswith('c'):     # continue
        stepping = False
        return True
    elif command.startswith('p'):     # print 
        if arg is None:
            print my_locals
        else:
            if arg in my_locals:
                print "{} = {}".format(arg, repr(my_locals[arg]))
            else:
                print "No such variable: {}".format(arg)
                return False
        return True
    elif command.startswith('b'):     # breakpoint  
        if arg is None:
            print "You must supply a line number"
            return False
        else:
            breakpoints[int(arg)] = True
            return True
    elif command.startswith('w'):     # watch variable
        if arg is None:
            print "You must supply a variable name"
            return False
        else:
            watchpoints[arg] = my_locals.get(arg)
            return True
    elif command.startswith('q'):     # quit
        print "Exiting my-spyder..."
        sys.exit(0)
    else:
        print "No such command", repr(command)  
    
    return False


commands = ["p", "s", "p tag", "p foo", "q"]

def input_command():
    command = raw_input("(my-spyder): ")
    #global commands
    #command = commands.pop(0)
    return command


def traceit(frame, event, trace_arg):
    """The local trace function should return a reference to itself 
    (or to another function for further tracing in that scope), 
    or None to turn off tracing in that scope."""
    global stepping
    global breakpoints
    global watchpoints

    if event == 'line':
        if stepping or breakpoints.has_key(frame.f_lineno):
            resume = False
            while not resume:
                print event, frame.f_lineno, frame.f_code.co_name, frame.f_locals
                command = input_command()
                resume = debug(command, frame.f_locals)
        else:
            for variable, value in watchpoints.items():
                if (variable in frame.f_locals):
                    current_value = frame.f_locals[variable]
                    if current_value != value:
                        print "Variable '{}' changed from {} to {}".format(variable, repr(value), repr(current_value))
                        watchpoints[variable] = current_value
                        resume = False
                        while not resume:
                            print event, frame.f_lineno, frame.f_code.co_name, frame.f_locals
                            command = input_command()
                            resume = debug(command, frame.f_locals)
                        break
    return traceit

# Using the tracer
sys.settrace(traceit)
main()
sys.settrace(None)

Variable 'out' changed from None to ''
line 9 remove_html_markup {'quote': False, 's': 'xyz', 'tag': False, 'out': ''}
(my-spyder): q
Exiting my-spyder...


SystemExit: 0